# Loading the Open COVID-19 Dataset
This very short notebook showcases how to load the [Open COVID-19 datset](https://github.com/open-covid-19/data), including some examples for commonly performed operations.

First, loading the data is very simple with `pandas`. We can use the CSV master table to download the entire Open COVID-19 dataset in a single step:

In [1]:
import pandas as pd

# Load CSV data directly from the URL with pandas, the options are needed to prevent
# reading of records with key "NA" (Namibia) as NaN
data = pd.read_csv(
    "https://storage.googleapis.com/covid19-open-data/v2/7/main.csv",
    keep_default_na=False,
    na_values=[""],
)

# NOTE: We are only downloading the last 7 days of data, but you can download all
# the data at once if you'd like by changing the URL to:
# https://storage.googleapis.com/covid19-open-data/v2/main.csvhttps://storage.googleapis.com/covid19-open-data/v2/main.csv


# Print a small snippet of the dataset
print(f"The dataset currently contains {len(data)} records, here is a sample:")
data.sample(5)

The dataset currently contains 44433 records, here is a sample:


,key,date,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,...,snowfall,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator
13168,HT_AR,2020-06-27,Q844024,NaN,HT,Haiti,AR,Artibonite,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2794,CO_05_05664,2020-06-26,Q1441754,NaN,CO,Colombia,05,Antioquia,5664,San Pedro De Los Milagros,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10632,ES,2020-06-24,Q29,country/ESP,ES,Spain,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33565,US_ND_38023,2020-06-27,Q28311,geoId/38023,US,United States of America,ND,North Dakota,38023,Divide County,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19626,PY_3,2020-06-28,Q755121,NaN,PY,Paraguay,3,Cordillera Department,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Looking at country-level data
Some records contain country-level data, in other words, data that is aggregated at the country level. Other records contain region-level data, which are subdivisions of a country; for example, Chinese provinces or USA states. A few regions also report at an even smaller subdivision, i.e. county/municipality level.

To filter only country-level data from the dataset, look for records that have a `aggregation_level == 0` or, alternatively, null value for the `subregion1_code` (or `subregion1_name`) field:

In [2]:
# Look for rows with country level data
# Same as `data[data.subregion2_code.isna()]`
countries = data[data.aggregation_level == 0]

# We no longer need the subregion-level columns
countries = countries.drop(columns=['subregion1_code', 'subregion1_name', 'subregion2_code', 'subregion2_name'])

countries.tail()

,key,date,wikidata,datacommons,country_code,country_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,school_closing,...,snowfall,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator
44428,ZW,2020-06-25,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44429,ZW,2020-06-26,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44430,ZW,2020-06-27,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44431,ZW,2020-06-28,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44432,ZW,2020-06-29,Q954,country/ZWE,ZW,Zimbabwe,ZW,ZWE,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Looking at state/province data
Conversely, to filter state/province data for a specific country, we need to look for records where the aggregation level is `1` (or where the region columns have non-null values). The following snippet extracts data related to Spain's subregions from the dataset:

In [3]:
# Filter records that have the right country code AND a non-null region code
# Same as `data[(data.country_code == 'ES') & ~(data.subregion`_code.isna())]`
spain_regions = data[(data.country_code == 'ES') & (data.aggregation_level == 1)]

# We no longer need the municipality-level columns
spain_regions = spain_regions.drop(columns=['subregion2_code', 'subregion2_name'])

spain_regions.tail()

,key,date,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,3166-1-alpha-2,3166-1-alpha-3,...,snowfall,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator
10732,ES_VC,2020-06-23,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10733,ES_VC,2020-06-24,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10734,ES_VC,2020-06-25,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10735,ES_VC,2020-06-26,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10736,ES_VC,2020-06-27,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,ES,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Using the `key` column
The `key` column is present in all datasets and is unique for each combination of country, province/state and municipality/county. This way, we can retrieve a specific country or region using a single filter for the data. The `key` column is built using `country_code` for country-level data, `${country_code}_${subregion1_code}` for province/state level data, and `${country_code}_${subregion1_code}_${subregion2_code}` for municipality/county data:

In [4]:
# Filter records for Spain at the country-level
spain_country = data[data.key == 'ES']

# We no longer need the subregion-level columns
spain_country = spain_country.drop(columns=['subregion1_code', 'subregion1_name', 'subregion2_code', 'subregion2_name'])

spain_country.tail()

,key,date,wikidata,datacommons,country_code,country_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,school_closing,...,snowfall,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator
10634,ES,2020-06-26,Q29,country/ESP,ES,Spain,ES,ESP,0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10635,ES,2020-06-27,Q29,country/ESP,ES,Spain,ES,ESP,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10636,ES,2020-06-28,Q29,country/ESP,ES,Spain,ES,ESP,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10637,ES,2020-06-29,Q29,country/ESP,ES,Spain,ES,ESP,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10638,ES,2020-06-30,Q29,country/ESP,ES,Spain,ES,ESP,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Filter records for Madrid, one of the subregions of Spain
madrid = data[data.key == 'ES_MD']

madrid.tail()

,key,date,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,...,snowfall,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator
10706,ES_MD,2020-06-23,Q5756,nuts/ES30,ES,Spain,MD,Comunidad de Madrid,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10707,ES_MD,2020-06-24,Q5756,nuts/ES30,ES,Spain,MD,Comunidad de Madrid,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10708,ES_MD,2020-06-25,Q5756,nuts/ES30,ES,Spain,MD,Comunidad de Madrid,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10709,ES_MD,2020-06-26,Q5756,nuts/ES30,ES,Spain,MD,Comunidad de Madrid,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10710,ES_MD,2020-06-27,Q5756,nuts/ES30,ES,Spain,MD,Comunidad de Madrid,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Dataset Subsets
The master table can be large and cumbersome depending on your application. If you only need a subset of the data, you can consult each table individually. For a list of all the available tables, see the [README](../README.md) of the repo. For example, here's how you would get only epidemiology data for Madrid:

In [6]:
# Load the epidemiology table
# Note that all the helper columns such as country code, country name, aggregation level, etc. are present in the
# `index` table; we only have the key here
epi = pd.read_csv('https://storage.googleapis.com/covid19-open-data/v2/epidemiology.csv')

# Filter records for Madrid, one of the subregions of Spain
madrid = epi[epi.key == 'ES_MD']

madrid.tail()

,date,key,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested
269883,2020-05-16,ES_MD,24.0,21.0,NaN,NaN,66663.0,8847.0,NaN,NaN
274513,2020-05-17,ES_MD,39.0,16.0,NaN,NaN,66702.0,8863.0,NaN,NaN
279114,2020-05-18,ES_MD,126.0,31.0,NaN,NaN,66828.0,8894.0,NaN,NaN
283809,2020-05-19,ES_MD,139.0,18.0,NaN,NaN,66967.0,8912.0,NaN,NaN
288476,2020-05-20,ES_MD,82.0,19.0,NaN,NaN,67049.0,8931.0,NaN,NaN


### Data consistency
Often, region-level data and country-level data will come from different sources. This will lead to numbers not adding up exactly, or even date misalignment (the data for the region may be reported sooner or later than the whole country). However, country- and region- level data will *always* be self-consistent